# Example about posting

In [ ]:
from atproto import Client, client_utils


def main():
    client = Client()
    profile = client.login('atlasover.bsky.social', 'STIG8roberts_alza')
    print('Welcome,', profile.display_name)

    text = client_utils.TextBuilder().text('Hello World from ').link('Python SDK', 'https://atproto.blue')
    post = client.send_post(text)
    client.like(post.uri, post.cid)

if __name__ == '__main__':
    main()


# Login

In [1]:
# 1. Login (Usa le tue credenziali Bluesky vere)
# Consiglio: crea una "App Password" dalle impostazioni di Bluesky per sicurezza
USERNAME = 'atlasover.bsky.social'
PASSWORD = 'STIG8roberts_alza'

In [2]:
from atproto import Client
import time

client = Client()
client.login(USERNAME, PASSWORD)
print(f"Loggato come: {USERNAME}")

Loggato come: atlasover.bsky.social


# Retrieving posts about a topic

In [ ]:
query = "venezuela"
limit = 10 
print(f"--- Cerco post con hashtag: {query} ---")
# Cerca i post. 'q' è la query string
search_results = client.app.bsky.feed.search_posts(params={'q': query, 'limit': limit})

seeds = []
for post in search_results.posts:
    print(f"Trovato post di {post.author.handle}: {post.record.text[:50]}...")
    seeds.append(post.uri) # L'URI è l'ID univoco del post che ci serve dopo
seeds

# Test Create a Net

## Test 

In [3]:
from atproto import Client
import time

# 2. Funzione per cercare post iniziali (I "Seed")
def get_seed_posts(query, limit=10):
    print(f"--- Cerco post con hashtag: {query} ---")
    # Cerca i post. 'q' è la query string
    search_results = client.app.bsky.feed.search_posts(params={'q': query, 'limit': limit, 'sort': 'top'})
    
    seeds = []
    for post in search_results.posts:
        # print(f"Trovato post di {post.author.handle}: {post.record.text[:50]}...")
        seeds.append(post.uri) # L'URI è l'ID univoco del post che ci serve dopo
    return seeds

# 3. Funzione per scaricare le risposte (Creare gli Archi)
def get_interaction_edges(post_uri):
    # Scarica il thread completo (post + risposte)
    # depth=1 significa che prendiamo solo le risposte dirette (A risponde a B)
    # Aumenta depth se vuoi le risposte delle risposte
    thread_data = client.get_post_thread(uri=post_uri, depth=1)
    
    if not thread_data.thread.replies:
        return []

    edges = []
    original_author = thread_data.thread.post.author.handle
    
    # Analizza chi ha risposto
    for reply in thread_data.thread.replies:
        # Controlliamo che sia una risposta valida e non un post cancellato
        if hasattr(reply, 'post'):
            reply_author = reply.post.author.handle
            
            # Creiamo l'arco: REPLY_AUTHOR -> ORIGINAL_AUTHOR
            edge = (reply_author, original_author)
            edges.append(edge)
            
    return edges

import pandas as pd
from atproto import exceptions

def get_thread_with_attributes(post_uri, max_retries=3):
    for attempt in range(max_retries):
        try:
            thread_data = client.get_post_thread(uri=post_uri, depth=1)
            print(f"thread_data {thread_data.thread.post.__dict__.keys()}")
            post = thread_data.thread.post.model_dump()
            if not thread_data.thread.replies:
                return [], {}
            
            edges = []
            users_info = {} # Qui salveremo gli attributi
            
            # 1. Dati dell'autore originale (Target)
            original_post = thread_data.thread.post
            original_author = original_post.author
            original_handle_clean = original_post.author.handle.replace('.bsky.social', '')
            profilo_completo = client.app.bsky.actor.get_profile(params={'actor': original_author.handle})
            
            print(f"Dati profilo: {profilo_completo.__dict__.keys()}")
            
            # Salviamo i dati dell'autore originale
            users_info[original_handle_clean] = {
                'followers': getattr(profilo_completo, 'followers_count', 0),
                'posts': getattr(profilo_completo, 'posts_count', 0),
            }
            
            # 2. Analisi delle risposte (Source)
            for reply in thread_data.thread.replies:
                if hasattr(reply, 'post'):
                    reply_author = reply.post.author
                    source_handle = reply_author.handle
                    source_handle_clean = reply_author.handle.replace('.bsky.social', '')
                    target_handle_clean = original_author.handle.replace('.bsky.social', '')
                    
                    # Creiamo l'arco
                    edges.append((source_handle_clean, target_handle_clean))
                    
                    profilo_completo = client.app.bsky.actor.get_profile(params={'actor': source_handle})
                    
                    # Salviamo i dati di chi risponde
                    users_info[source_handle_clean] = {
                        'followers': getattr(profilo_completo, 'followers_count', 0),
                        'posts': getattr(profilo_completo, 'posts_count', 0),
                    }
                
            return edges, users_info
            
        except (exceptions.InvokeTimeoutError, TimeoutError) as e:
            if attempt < max_retries - 1:
                print(f"⚠️ Timeout al tentativo {attempt + 1}/{max_retries}. Riprovando tra 5 secondi...")
                time.sleep(5)
            else:
                print(f"❌ Fallito dopo {max_retries} tentativi. Saltando questo post.")
                return [], {}
        except Exception as e:
            print(f"❌ Errore: {type(e).__name__}. Saltando questo post.")
            return [], {}

In [ ]:
# --- ESECUZIONE AGGIORNATA CON GESTIONE ERRORI ---

seed_uris = get_seed_posts('#venezuela', limit=1)

all_edges = []
all_users_data = {}

print("--- Inizio raccolta dati arricchiti ---")
print(f"Tot post da elaborare: {len(seed_uris)}\n")

for i, uri in enumerate(seed_uris, 1):
    print(f"[{i}/{len(seed_uris)}] Elaborazione post...")
    edges, users = get_thread_with_attributes(uri, max_retries=3)
    all_edges.extend(edges)
    all_users_data.update(users)
    time.sleep(3)  # Delay aumentato a 3 secondi tra le richieste

# --- CREAZIONE GRAFO CON ATTRIBUTI ---
import networkx as nx

G = nx.DiGraph()
G.add_edges_from(all_edges)

# LA MAGIA: Assegniamo gli attributi ai nodi
for node_id in G.nodes():
    if node_id in all_users_data:
        attributes = all_users_data[node_id]
        G.nodes[node_id]['followers'] = attributes['followers']
        G.nodes[node_id]['posts'] = attributes['posts']

print(f"\n✅ Grafo creato: {G.number_of_nodes()} nodi, {G.number_of_edges()} archi.")

# Salvataggio
nx.write_gexf(G, "rete_venezuela_arricchita.gexf")
print("✅ File salvato: rete_venezuela_arricchita.gexf")

# save to csv file

In [ ]:
import networkx as nx

# 1. Creiamo un oggetto Grafo di NetworkX
G = nx.DiGraph() # DiGraph significa "Grafo Diretto" (A risponde a B è diverso da B risponde ad A)

# 2. Aggiungiamo gli archi dalla tua lista
# La lista deve essere tipo [('utente_che_risponde', 'autore_post'), ...]
G.add_edges_from(rete_sociale)

print(f"Il grafo ha {G.number_of_nodes()} nodi e {G.number_of_edges()} archi.")

# 3. Salviamo in formato GEXF (Ottimo per Gephi)
nx.write_gexf(G, "rete_venezuela.gexf")

# 4. Salviamo anche in CSV semplice (Lista Archi)
import csv
with open("rete_venezuela.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Source", "Target"]) # Intestazione che piace a Gephi
    writer.writerows(rete_sociale)

print("File salvati! Ora puoi aprirli con Gephi.")

# Get follower count by a profile

In [ ]:
import pandas as pd

res = client.app.bsky.actor.get_profile(params={'actor': 'taggartrehnnauthor.bsky.social'})
res2 = pd.json_normalize(res.model_dump())
res2['follows_count']
res2['followers_count']